**Status: The qcodes dataset changes are still fresh. Proceed at your device's risk.**

# Installation
From [the docs](http://qcodes.github.io/Qcodes/start/index.html):

1. Install [Anaconda](https://www.anaconda.com/download/). Get the Python 3 version. No need to install the VSCode stuff.
1. Save [environment.yml](https://raw.githubusercontent.com/QCoDeS/Qcodes/master/environment.yml) to the machine.
1. Run the following in an Anaconda Prompt:
```
conda env create -f environment.yml
activate qcodes
pip install qcodes
```
1. Now you can run `jupyter notebook` from within your qcodes environment.

In [ ]:
%matplotlib notebook
import time
import numpy as np
import matplotlib.pyplot as plt
import qcodes as qc
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_export import get_data_by_id

The following is not standard and I just use it for live plotting. In the future, lets replace it with something better!

In [ ]:
def create_1d_plots(meas):
    xparams = []
    yparams = []
    for k in meas.parameters:
        if meas.parameters[k].depends_on or meas.parameters[k].inferred_from:
            yparams.append(meas.parameters[k])
        else:
            xparams.append(meas.parameters[k])
    fig, axs = plt.subplots(len(xparams), len(yparams), figsize=(4*len(yparams), 2*len(xparams)), squeeze=False)
    for axx, xp in zip(axs, xparams):
        for ax, yp in zip(axx, yparams):
            ax.set_xlabel(f'{xp.label} ({xp.unit})')
            ax.set_ylabel(f'{yp.label} ({yp.unit})')
    lines = [[ax.plot([], [])[0] for ax in axx] for axx in axs]
    fig.show()
    return fig, axs, lines


def update_1d_plots(results, length, state):
    if not results:
        return
    fig, axs, lines = state
    values = list(zip(*results))
    for i, ls in enumerate(lines):
        for j, l in enumerate(ls):
            l.set_xdata(np.append(l.get_xdata(), values[i]))
            l.set_ydata(np.append(l.get_ydata(), values[j + len(lines)]))
    for axx in axs:
        for ax in axx:
            ax.relim()
            ax.autoscale_view()
    fig.tight_layout()
    fig.canvas.draw()

# Instrument setup

At this point, each measurement setup needs to define which instruments they use. See [the docs](http://qcodes.github.io/Qcodes/user/intro.html#instrument). For demonstration purposes, here I just use dummy instruments `dac` and `dmm`.

In [ ]:
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument(name="dac", gates=['ch1', 'ch2'])
dmm = DummyInstrument(name="dmm", gates=['idc', 'ig'])

import random
dmm.idc.get = lambda: random.gauss(1, 0.1)
dmm.ig.get = lambda: random.gauss(0, 0.01)

You can set `step` and `inter_delay` on any parameters. These will apply to all future attempts to set the value, including those in loops. For instance, if `dac.ch1` can be ramped at a maximum speed of 0.1 V / ms, then do this:

In [ ]:
dac.ch1.step = 0.1
dac.ch1.inter_delay = 0.001

# Experiment setup

The first time you run qcodes on a new computer, you need to create the SQLite database. See [the docs](https://qcodes.github.io/Qcodes/dataset/index.html) for details on the design of the dataset and database. You will then want to make a new experiment to hold your data. At the very least, make a new experiment when you change out your samples.

In [ ]:
qc.initialise_database()
qc.new_experiment(name='demo', sample_name='my best sample')

# Measurement

## 1D sweep

In its most basic form, a 1D sweep looks like this. Below, I will go into more details.

In [ ]:
meas = Measurement()
meas.register_custom_parameter('x')
meas.register_custom_parameter('y', setpoints=('x',))

with meas.run() as datasaver:
    for x in np.linspace(0, 25, 10):
        # Set x.
        # Measure y.
        datasaver.add_result(('x', x), ('y', 0.0))

We use a `Measurement` object to specify which parameters are dependent and which are independent. In this example, I define two of each. Note that the `'time'` parameter is not a `qcodes.Parameter`!

In [ ]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_custom_parameter('time', label='Time', unit='s')
meas.register_parameter(dmm.ig, setpoints=(dac.ch1, 'time',))
meas.register_parameter(dmm.idc, setpoints=(dac.ch1, 'time',))

A measurement's `write_period` specifies how often to write to the database, in seconds. This also affects how often to plot.

In [ ]:
meas.write_period = 1

Add actions and after the run as necessary. This is useful for things like sweeping up gates, ensuring the magnet is off after the run, and so on.

In [ ]:
meas.add_before_run(lambda dac: dac.ch2.set(10), (dac,))
meas.add_after_run(lambda dac: dac.ch2.set(0), (dac,))

`meas.run()` will give you a nice context manager which you can use to save data. I like to catch `KeyboardInterrupt`, which will be sent when the stop button is pressed in jupyter.

In [ ]:
meas.add_subscriber(update_1d_plots, state=create_1d_plots(meas))
with meas.run() as datasaver:
    try:
        time.sleep(5)
        t0 = time.monotonic()
        for set_v in np.linspace(0, 25, 10):
            dac.ch1.set(set_v)
            time.sleep(1)
            datasaver.add_result(
                (dac.ch1, set_v),
                (dmm.ig, dmm.ig.get()),
                (dmm.idc, dmm.idc.get()),
                ('time', time.monotonic() - t0))
    except KeyboardInterrupt:
        pass

## 2D sweep

TODO: Estimate total time. Live plotting.

In [ ]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.ig, setpoints=(dac.ch1, dac.ch2))

with meas.run() as datasaver:
    try:
        for set_ch1 in np.linspace(0, 25, 10):
            for set_ch2 in np.linspace(0, 10, 10):
                dac.ch1.set(set_ch1)
                dac.ch2.set(set_ch2)
                ig = dmm.ig.get()

                datasaver.add_result((dac.ch1, set_ch1), (dac.ch2, set_ch2), (dmm.ig, ig))
    except KeyboardInterrupt:
        pass

## Where is the data?

See the `get_data_by_id` docstring.

TODO: Make this easier to save to a csv or something.

In [ ]:
get_data_by_id(datasaver.run_id)